In [1]:
import numpy as np
import urllib
from sklearn import preprocessing
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from numpy import loadtxt, where
from pylab import scatter, show, legend, xlabel, ylabel
from matplotlib.colors import ListedColormap
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

In [2]:
#Preprocessed clean version of input file
input_file = "Cleaned_file.csv"

In [3]:
df = pd.read_csv(input_file)

In [4]:
# Printing Data Shape
df.shape

(213464, 6)

In [5]:
df.head()

,District,Date,Primary Type,Arrest,Crime Weightage,Label
0,11,05/03/2016 09:00:00 PM,NARCOTICS,True,5,1
1,8,05/03/2016 09:35:00 PM,THEFT,False,1,1
2,9,05/03/2016 02:45:00 PM,ASSAULT,False,3,0
3,8,05/03/2016 06:42:00 PM,BATTERY,False,2,1
4,3,05/03/2016 03:00:00 PM,ROBBERY,False,1,0


In [6]:
#dataset = np.loadtxt(input_file, delimiter=",")

In [7]:
# Giving numeric values for text data
primaryType = preprocessing.LabelEncoder()
arrest = preprocessing.LabelEncoder()
#date = preprocessing.LabelEncoder()

In [8]:
df["Primary Type"] = primaryType.fit_transform(df["Primary Type"])
df["Arrest"] = arrest.fit_transform(df["Arrest"])
#df["Date"] = date.fit_transform(df["Date"])

In [9]:
df['Date'] = df['Date'].apply(lambda x: x.split('/')[0])

In [10]:
#df['Date'] = df['Date'].dt.month

In [11]:
df.head()

,District,Date,Primary Type,Arrest,Crime Weightage,Label
0,11,05,17,1,5,1
1,8,05,31,0,1,1
2,9,05,1,0,3,0
3,8,05,2,0,2,1
4,3,05,28,0,1,0


In [12]:
#df["Date"] = date.inverse_transform(df["Date"])

In [13]:
#df.head()

In [14]:
array = df.values

In [15]:
#columns = ["District", "Date", "Primary Type", "Arrest", "Crime Weightage"]

In [16]:
# Features
X = list(array[:,0:5])

In [17]:
# Target
Y = list(array[:,5])

In [18]:
totalTarget = df["Label"].count()

In [19]:
# Count of 0s in the target
zeroCount = df[(df["Label"] == 0)].count()[0]

In [20]:
# Count of 1s in the target
oneCount = df[(df["Label"] == 1)].count()[0]

In [21]:
#Probability of 0s in the target
probZero = zeroCount / totalTarget

In [22]:
#Probability of 1s in the target
probOne = oneCount / totalTarget

In [23]:
print("Probability of 0s in the Target ", '%0.3f'%(probZero))

Probability of 0s in the Target  0.592


In [24]:
print("Probability of 1s in the Target ", '%0.3f'%(probOne))

Probability of 1s in the Target  0.408


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.67, random_state=3)

In [26]:
clf = LogisticRegression(random_state=2)

In [27]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
Y_pred    = clf.predict(X_test)
Y_score   = clf.decision_function(X_test)
accuracy  = metrics.accuracy_score(y_test, Y_pred)
precision = metrics.precision_score(y_test, Y_pred)
recall    = metrics.recall_score(y_test, Y_pred)
AUC       = metrics.roc_auc_score(y_test, Y_score)

In [29]:
print('%0.3f'%accuracy,'%0.3f'%precision, '%0.3f'%recall,'%0.3f'%AUC)

0.692 0.674 0.488 0.654


In [30]:
scores = cross_val_score(clf,X_train,y_train,cv=10,scoring='accuracy')

In [31]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.70 (+/- 0.01)


In [ ]:
MLPC = MLPClassifier(random_state=2)

In [ ]:
MLPC.fit(X_train,y_train)

In [ ]:
scores = cross_val_score(MLPC,X_train,y_train,cv=10,scoring='accuracy')

In [ ]:
scores = cross_val_score(clf,X_train,y_train,cv=10,scoring='accuracy')